# Content-Aware Image Retargeting
Reduce the width of images (`Baby.png`, `Diana.png`, `Snowman.png`) by `seams_number` columns using Seam Carving, implemented from scratch. Optionally use depth/saliency maps or other energy functions. Visualize seams if enabled.

In [105]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [106]:
def seam_carving(image, seams_number, depth_map=None, saliency_map=None, visualize=False):
    """Perform Seam Carving to reduce image width by seams_number columns.

    Args:
        image: Input image (H, W, 3) as float32 in [0, 1].
        seams_number: Number of vertical seams to remove.
        depth_map: Optional depth map (H, W) as float32.
        saliency_map: Optional saliency map (H, W) as float32.
        visualize: If True, display seams during processing.

    Returns:
        resized_image: Image with reduced width.
        seam_energies: List of cumulative energies of removed seams.
    """
    # TODO: Implement Seam Carving from scratch
    # 1. Compute energy map (e.g., gradient, depth, or saliency-based)
    seam_energies = []
    for index in range(seams_number):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        energy = np.abs(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)) + np.abs(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3))
        if depth_map is not None:
            depth_resized = cv2.resize(depth_map, (energy.shape[1], energy.shape[0]))
            energy += depth_resized * 5.0
        if saliency_map is not None:
            saliency_resized = cv2.resize(saliency_map, (energy.shape[1], energy.shape[0]))
            energy += saliency_resized * 10.0
    # 2. Find lowest-energy vertical seam using dynamic programming
        h, w = energy.shape
        dp = energy.copy()
        path = np.zeros_like(dp, dtype=np.int32)
        for i in range(1, h):
            for j in range(0, w):
              min_value = dp[i-1, j]
              min_index = j
              if(j-1 >= 0 and min_value > dp[i-1, j-1]):
                min_value = dp[i-1, j-1]
                min_index = j-1
              if(j+1 < w and min_value > dp[i-1, j+1]):
                min_value=dp[i-1, j+1]
                min_index = j+1
              dp[i, j] += min_value
              path[i, j] = min_index
        seam = []
        j = np.argmin(dp[-1])
        seam.append(j)
        total_energy = dp[-1, j]
        for i in range(h-1, 0, -1):
            j = path[i, j]
            seam.append(j)
            total_energy += energy[i, j]
        seam.reverse()
        seam_energies.append(total_energy)
    # 3. Remove seam and update image
        h, w, c = image.shape
        new_img = np.zeros((h, w-1, c), dtype=np.float32)
        for i in range(h):
            j_seam = seam[i]
            new_img[i, :, :] = np.concatenate([image[i, :j_seam, :], image[i, j_seam+1:, :]], axis=0)
        image = new_img
    # 4. Repeat for seams_number iterations
    # 5. Visualize seams if flag enabled
        if visualize:
            img = image.copy()
            for i in range(h):
                img[i, seam[i]] = [1.0, 0.0, 0.0]
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f'Seam :{index}')
            plt.axis('off')
            plt.show()
    return image, seam_energies

In [107]:
# Load images and maps
baby_path = './images_CAIR/Baby.png'
baby_dmap_path = './images_CAIR/Baby_DMap.png'
baby_smap_path = './images_CAIR/Baby_SMap.png'

baby = cv2.imread(baby_path).astype(np.float32) / 255.0
baby_dmap = cv2.imread(baby_dmap_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
baby_smap = cv2.imread(baby_smap_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0

# Parameters
seams_number = 100
visualize = True

In [ ]:
# Process images
baby_resized, baby_energies = seam_carving(baby, seams_number, baby_dmap, baby_smap, visualize)

# Save seam energies
with open('seam_energy_log.txt', 'w') as f:
    f.write('Baby Seam Energies:\n' + '\n'.join(map(str, baby_energies)) + '\n')

# Visualize results
plt.imshow(cv2.cvtColor(baby_resized, cv2.COLOR_BGR2RGB))
plt.title('Baby Resized')
plt.axis('off')
plt.show()

# Save resized images
cv2.imwrite('Baby_resized.png', (baby_resized * 255).astype(np.uint8))